In [1]:
import autograd.numpy as np
from autograd import grad


In [2]:
nside=256
freqs=np.arange(544,1088,step=1)
# nfreqs=len(freqs)
nfreqs=181
lmax=3*nside -1
r=3 #foreground model rank
nbins=76

In [3]:
x=np.load('/home/iloveorcas/MASS_Thesis/results/ff_maps.npy')

In [5]:
noise_binned= np.load('/home/iloveorcas/MASS_Thesis/results/noise_binned.npy')
hi_binned= np.load('/home/iloveorcas/MASS_Thesis/results/hi_binned.npy')
c_hat=np.load('/home/iloveorcas/MASS_Thesis/results/binned_empirical.npy')
print(hi_binned.shape)

(76, 181, 181)


In [8]:

def smica_new(params,hi, noise, empirical,  n_bins, nfreqs, r):
    
    # Reshape the flattened params back into F_init and P_b_init
    
    F = params[:nfreqs * r].reshape((nfreqs, r))
    P_b = params[nfreqs * r:].reshape((n_bins,r,r))
    
    
    """# === Normalize columns of F and rescale P accordingly ===
    for i in range(r):
        norm = np.linalg.norm(F[:, i])

        F[:, i] /= norm  # normalize column
        for b in range(n_bins):
            P_b[b, i, :] *= norm
            P_b[b, :, i] *= norm  # symmetric scaling"""
    
    
    # norm = np.linalg.norm(F, axis=0)
    
    # # Normalize F
    # F_normalized = F / norm        
    # # Adjust P_b: D^{-1} P_b D^{-1}
    # D_inv = np.diag(1.0 / norm)
    # for b in range(n_bins):
    #     P_b[b] = D_inv @ P_b[b] @ D_inv
                
    # === Compute the cost function ===            
    cost=0
    # model=np.zeros((n_bins, nfreqs, nfreqs))
    # model_inv=np.zeros((n_bins, nfreqs, nfreqs))
    
    # term=np.zeros(n_bins)
    for b in range(n_bins):
        model= hi[b] + noise[b] + np.dot(F, np.dot(P_b[b], F.T))
        model_inv= np.linalg.inv(model)
        
        sign, logdet= np.linalg.slogdet(np.dot(empirical[b],model_inv))
        trace=np.trace(np.dot(empirical[b] , model_inv))
        
        term = logdet + trace -nfreqs
        cost += term
        
    return cost




In [9]:
def pca_new(empirical, r, n_bins):
    """
    Perform PCA on the covariance matrix.
    
    Parameters:
    ----------
    c_hat : np.ndarray
        Covariance matrix of shape (n_bins, n_freq, n_freq).
    
    r : int
        Number of principal components to retain.
    
    Returns:
    -------
    F : np.ndarray
        Matrix of shape (n_freq, r) containing the principal components.
    
    P_b : np.ndarray
        Matrix of shape (n_bins, r, r) containing the eigenvalues and eigenvectors.
    """
    # Perform PCA
    R_global = np.mean(empirical, axis=0)  # c_hat has shape (n_bins, n_freqs, n_freqs)
    #eigen decompositiom
    eigvals, eigvecs = np.linalg.eigh(R_global) #ascending order
    #Take the top r eigenvectors:
    F = eigvecs[:, -r:]  # largest eigenvectors

    P_b = np.zeros((n_bins, r, r))
    for b in range(n_bins):
        P_b[b] =np.dot( F.T , np.dot(empirical[b] , F))
        P_b[b] = 0.5 * (P_b[b] + P_b[b].T)
    return eigvals,F, P_b

In [10]:
_,F_pca_new, P_b_pca_new = pca_new(c_hat,r, nbins)
print(F_pca_new.shape) 
print(P_b_pca_new.shape)
params_pca = np.concatenate([F_pca_new.flatten(), P_b_pca_new.flatten()])
auto_grad=grad(smica_new,argnum=0)
answer=auto_grad(params_pca,hi_binned, noise_binned, c_hat, nbins, nfreqs, r)

print(answer)
print(answer.shape)

(181, 3)
(76, 3, 3)
[  -86.77678482   -21.32990707  -516.09544222 ...   -74.84042524
  -272.96553058 -2088.48522816]
(1227,)


In [11]:
phi=smica_new(params_pca, hi_binned, noise_binned, c_hat, nbins, nfreqs, r)
print(phi)

-1707.294929961676


In [12]:
def taylor_sine(x,y):  # Taylor approximation to sine function
    ans = currterm = x
    i = 0
    while np.abs(currterm) > 0.001:
        currterm = -currterm * x**2*y**2 / ((2 * i + 3) * (2 * i + 2))
        ans = ans + currterm
        i += 1
    return ans

grad_sine = grad(taylor_sine)
print ("Gradient of sin(pi) is", grad_sine(np.pi, 1))

Gradient of sin(pi) is -0.9998995297042174


In [9]:
def taylor_sine(x,y):  # Taylor approximation to sine function
    ans = currterm = x
    i = 0
    while np.abs(currterm) > 0.001:
        currterm = -currterm * x**2 / ((2 * i + 3) * (2 * i + 2))
        ans = ans + currterm
        i += 1
    return ans*y

grad_sine = grad(taylor_sine)
print ("Gradient of sin(pi) is", grad_sine(np.pi,2))

Gradient of sin(pi) is -1.9997990594084347


In [13]:
f = lambda x: smica_new(x,hi=hi_binned, noise=noise_binned, empirical=c_hat, n_bins=nbins, nfreqs=nfreqs, r=r)
grad_f = lambda x: auto_grad(x,hi=hi_binned, noise=noise_binned, empirical=c_hat, n_bins=nbins, nfreqs=nfreqs, r=r)

In [28]:
def taylor_test(cost, grad_cost, params, epsilons=None):
    if epsilons is None:
        epsilons = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5] #stepsize

    np.random.seed(0)
    h = np.random.randn(*params.shape)
    h /= np.linalg.norm(h)  #unit vector

    # f0 = cost(params) #at a point x0
    f0=phi
    # grad0 = grad_cost(params) 
    grad0 =answer
    directional_derivative = np.dot(grad0.T, h)

    for eps in epsilons:
        f_eps = cost(params + eps * h)
        residual = abs(f_eps - f0 - eps * directional_derivative)
        sum= np.abs(f_eps - f0)
        print(f"epsilon = {eps:.1e}, Residual = {residual:.2e}, Residual/epsilon = {residual/eps:.2e}")
        print(sum)

taylor test for autograd

In [29]:
taylor_test(f, grad_f, params_pca)

epsilon = 1.0e-01, Residual = 8.66e+03, Residual/epsilon = 8.66e+04
7878.158988054337
epsilon = 1.0e-02, Residual = 8.07e+01, Residual/epsilon = 8.07e+03
158.79975161790276
epsilon = 1.0e-03, Residual = 5.46e+00, Residual/epsilon = 5.46e+03
2.351776621896761
epsilon = 1.0e-04, Residual = 3.12e+00, Residual/epsilon = 3.12e+04
2.3418619306130495
epsilon = 1.0e-05, Residual = 2.67e+00, Residual/epsilon = 2.67e+05
2.752051278647059


In [ ]:
# from scipy.optimize import approx_fprime
# approx_grad = approx_fprime(params_pca, f, epsilon=1e-6) #finite differences
# print(approx_grad) #4mins
# #took 14 mins on yan's pc

KeyboardInterrupt: 

In [ ]:
fin_diff=np.load('approx_grad.npy') #from yan

In [16]:
fin_diff

array([-6.44574911e+05,  1.06312433e+06,  2.66904942e+06, ...,
       -7.48378766e+01, -2.72929322e+02, -2.08683645e+03], shape=(1227,))

In [18]:
answer

array([  -86.77678482,   -21.32990707,  -516.09544222, ...,
         -74.84042524,  -272.96553058, -2088.48522816], shape=(1227,))

In [22]:
grad_approx = lambda x: approx_fprime(x,f=f, epsilon=1e-6)  # finite differences

Taylor test with fin diff method

In [30]:
taylor_test(f, grad_approx, params_pca)

epsilon = 1.0e-01, Residual = 8.66e+03, Residual/epsilon = 8.66e+04
7878.158988054337
epsilon = 1.0e-02, Residual = 8.07e+01, Residual/epsilon = 8.07e+03
158.79975161790276
epsilon = 1.0e-03, Residual = 5.46e+00, Residual/epsilon = 5.46e+03
2.351776621896761
epsilon = 1.0e-04, Residual = 3.12e+00, Residual/epsilon = 3.12e+04
2.3418619306130495
epsilon = 1.0e-05, Residual = 2.67e+00, Residual/epsilon = 2.67e+05
2.752051278647059
